In [ ]:
import pandas as pd
import numpy as np
import re
#import required functions
import warnings
warnings.filterwarnings("ignore")
import math
import os
from pandas import ExcelWriter
from pandas import ExcelFile
import datetime
from pandas import DataFrame
import itertools
#from sklearn.preprocessing import StandardScaler
import random
import torch
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from bs4 import BeautifulSoup
import mechanize
import urllib.request
import http.cookiejar as cookielib
import requests
from pprint import pprint
import random

In [ ]:
def createfiledirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

# **Analyze Disneyland Reviews Raw Dataset Column Attributes**

In [ ]:
username='joythompson'
projdir="/Users/"+username+"/Desktop/DSE241_FinalProject_JoyT/"
tabraw=createfiledirectory(projdir + 'PreprocessedData/')
tabready=createfiledirectory(projdir + 'ProcessedTableauReady/')

reviews=pd.read_csv('https://github.com/JoyT10294/DSE241_FinalProject_JoyT/blob/main/PreprocessedData/DisneylandReviews.csv')
print(reviews)
numofregion=len(reviews.groupby(['Reviewer_Location']).count().reset_index())
maxrevlen=0
maxloclen=0
minrevlen=500
minloclen=5
for ind in reviews.index:
    if len(str(reviews.loc[ind, 'Review_Text']))>maxrevlen:
        maxrevlen=len(str(reviews.loc[ind, 'Review_Text']))
    elif len(str(reviews.loc[ind, 'Review_Text']))!=0 and len(str(reviews.loc[ind, 'Review_Text']))<minrevlen:
        minrevlen=len(str(reviews.loc[ind, 'Review_Text']))
    if len(str(reviews.loc[ind, 'Reviewer_Location']))>maxloclen:
        maxloclen=len(str(reviews.loc[ind, 'Reviewer_Location']))
    elif len(str(reviews.loc[ind, 'Reviewer_Location']))!=0 and len(str(reviews.loc[ind, 'Reviewer_Location']))<minloclen:
        minloclen=len(str(reviews.loc[ind, 'Reviewer_Location']))
print('# of Unique Countries: '+str(numofregion)+' Max Char Review Text: '+str(maxrevlen)+' Max Char Location: '+
      str(maxloclen)+' Min Char Review Text: '+str(minrevlen)+' Min Char Location: '+str(minloclen))

# **Extract Monthly Temperature & Humidity Historical Data from Timeanddate.com for All Branch Locations**

In [ ]:
def getweatherdata(linkloc, branchdata, country):  
    for year in range(2010, 2020):
        for month in range(1, 13):
            month=str(month)
            year=str(year)
            url=linkloc + month + '&year='+ year
            response =urllib.request.Request(url)
            result = urllib.request.urlopen(response)
            resulttext = result.read()
            soup = BeautifulSoup(resulttext)
            for script in soup(["script", "style"]):
                script.extract()    # rip it out
            text = soup.get_text()
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = '\n'.join(chunk for chunk in chunks if chunk)
            temptext=text.split('TemperatureHumidityPressureHigh')[1].split('Reported')[0]
            temphigh=temptext.split('°')[0]
            humidhigh=temptext.split(')')[1].split('%')[0]
            templow=temptext.split('Low')[1].split('°')[0]
            humidlow=temptext.split('Low')[1].split(')')[1].split('%')[0]
            tempavg=temptext.split('Average')[1].split('°')[0]
            humidavg=temptext.split('Average')[1].split('°F')[1].split('%')[0]
            if int(year)>2010 or int(month)>9:
                branchdata=branchdata.append({'location':country, 'month':month, 'year':year,'temp_high':temphigh, 'temp_low':templow, 'temp_average':tempavg,
                                          'humid_high':humidhigh,  'humid_low':humidlow, 'humid_average':humidavg}, ignore_index=True)
    return branchdata

In [ ]:
branchdata=pd.DataFrame(columns=['location','month', 'year' 'temp_high', 'temp_low', 'temp_average', 'humid_high', 'humid_low', 'humid_average'])
branchdata=getweatherdata('https://www.timeanddate.com/weather/france/paris/historic?month=', branchdata, 'Paris')
branchdata=getweatherdata('https://www.timeanddate.com/weather/usa/anaheim/historic?month=', branchdata, 'Anaheim')
branchdata=getweatherdata('https://www.timeanddate.com/weather/@1818609/historic?month=', branchdata, 'Hong Kong')
branchdata.to_csv(tabraw+'BranchMonthlyWeatherData.csv')

# **Append Monthly Temperature & Humidity Data According to Branch & Visit Month to Reviews Dataset**

In [ ]:
def addtemphumid(reviews, branchdata, city):  
    data=branchdata[branchdata['location']==city].set_index('year_mon')
    reviews.loc[ind, 'location']=city
    reviews.loc[ind, 'temp_average']=data.loc[yearmon, 'temp_average']
    reviews.loc[ind, 'temp_low']=data.loc[yearmon, 'temp_low']
    reviews.loc[ind, 'temp_high']=data.loc[yearmon, 'temp_high']
    reviews.loc[ind, 'humid_average']=data.loc[yearmon, 'humid_average']
    reviews.loc[ind, 'humid_low']=data.loc[yearmon, 'humid_low']
    reviews.loc[ind, 'humid_high']=data.loc[yearmon, 'humid_high']
    return reviews

In [ ]:
branchdata=pd.read_csv(tabraw+'BranchMonthlyWeatherData.csv')
for ind in branchdata.index:
    branchdata.loc[ind, 'year_mon']=str(branchdata.loc[ind,'year'])+ '-'+str(branchdata.loc[ind,'month'])
droplist=[]
for ind in reviews.index:
    yearmon=reviews.loc[ind, 'Year_Month']
    if str(yearmon) in ['2010-9', '2010-8','2010-7', '2010-6', '2010-5', 'missing','2010-4','2010-3','2010-2','2010-1']:
        droplist.append(ind)
    elif 'Paris' in reviews.loc[ind, 'Branch']:
        reviews=addtemphumid(reviews, branchdata, 'Paris')
    elif 'HongKong' in reviews.loc[ind, 'Branch']:
        reviews=addtemphumid(reviews, branchdata, 'Hong Kong')
    elif 'California' in reviews.loc[ind, 'Branch']:
        reviews=addtemphumid(reviews, branchdata, 'Anaheim')
review=reviews.drop(dropd
review.to_csv(tabready+'DisneyReviewerAppendWeatherData.csv')

# **Create Dataset of Top Unique Keywords/Phrases from Reviewers Comments**

In [ ]:
nlp = spacy.load("en_core_web_sm")
wordlabelpd=pd.DataFrame(columns=['Review_ID', 'EntityOrderID', 'Text', 'CategoryLabel']);
for ind in review.index:
    doc = nlp(str(review.loc[ind,'Review_Text']))
    entorder=0;
    for ent in doc.ents:
        entorder=entorder+1;
        label=ent.label_;
        wordlabelpd=wordlabelpd.append({'Review_ID':review.loc[ind, 'Review_ID'],
                'EntityOrderID':entorder , 'Text': str(ent.text).lower(), 'CategoryLabel': label}, ignore_index=True);

uniquepd=wordlabelpd.groupby(['Text']).count().reset_index().sort_values(by=['Review_ID'], ascending=False)
uniquepd.to_csv(tabraw+'ExtractWordsPhrasesReviewComment.csv')
wordlabelpd.to_csv(tabraw+'CreateUniqueWordListExtractedReviewComment.csv')

# **Create Dataset of All Cases a Top Unique Keyword/Phrases Exists in a Rewiewers Comments**

In [ ]:
#File uniquepd.csv manually cleaned and limited to 100 with excel to create top 100 words.csv
#Ideally string matching & data cleaning functions should be utilized to create the top word list
top=pd.read_csv(tabraw+'top 100 words.csv').drop(columns=['Unnamed: 0'])
review=pd.read_csv(tabraw+'DisneyReviewerAppendWeatherData.csv')
topwordall=pd.DataFrame(columns=['Top Word','Work Top Ranking', '# of Reviews w/Word','Review_ID', 'Rating', 'Year_Month', 'Reviewer_Location', 
       'Branch', 'location', 'temp_average', 'temp_low', 'temp_high','humid_average', 'humid_low', 'humid_high'])

for ind in top.index:
    text=top.loc[ind, 'Text']
    for i in range(1,6):
        top.loc[ind, 'Rating '+ str(i)]=0
    numofrate=0
    sumrate=0
    for ind2 in review.index:
        revtext=review.loc[ind2, 'Review_Text']
        rating=review.loc[ind2, 'Rating']
        if text in revtext:
            topwordall=topwordall.append({'Top Word':text, '# of Reviews w/Word':top.loc[ind, '# of Reviews w/Word'],
                                          'Work Top Ranking':ind+1, 'Review_ID': review.loc[ind2, 'Review_ID'], 
                                          'Rating':rating, 'Year_Month':review.loc[ind2, 'Year_Month'], 
                                          'Reviewer_Location':review.loc[ind2, 'Reviewer_Location'], 
                                           'Branch':review.loc[ind2, 'Branch'], 'location':review.loc[ind2, 'location'],
                                          'temp_average':review.loc[ind2, 'temp_average'],'temp_low':review.loc[ind2, 'temp_low'], 
                                          'temp_high': review.loc[ind2, 'temp_high'],'humid_average':review.loc[ind2, 'humid_average'],'humid_low':review.loc[ind2, 'humid_low'], 
                                          'humid_high': review.loc[ind2, 'humid_high']
                                         }, ignore_index=True)
            numofrate=numofrate+1
            sumrate=sumrate+rating
            top.loc[ind, 'Rating '+ str(int(rating))]=top.loc[ind, 'Rating '+ str(int(rating))]+1
    top.loc[ind, 'Rating Average']=np.round(sumrate/numofrate, decimals=2)
top.to_csv(tabready+'top_100_words_with_avg_rating.csv')
topwordall.to_csv(tabready+'top_100_words_with_all_review_data.csv')